# 개요

# Prepare

# 1. 초기 세팅

## Library & data load

In [163]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [164]:
# 처리용 df와 오리지널 버전 남겨놓음
df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202104.csv",encoding='euc_kr')
org_df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202104.csv",encoding='euc_kr')

# Pre-Processing

## Basic info

In [165]:
df.head()

,품목명,단위,등급,가격,산지,친환경구분,입력일
0,[쪽파]쪽파,1kg,특(1등),2400,전남 보성군,일반,20210331
1,[쪽파]쪽파,1kg,특(1등),2500,충남 아산시,일반,20210331
2,[쪽파]쪽파,1kg,특(1등),2600,충남 아산시,일반,20210331
3,[쪽파]쪽파,1kg,특(1등),2300,전남 보성군,일반,20210331
4,[쪽파]쪽파,1kg,특(1등),2700,전남 보성군,일반,20210331


In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466088 entries, 0 to 466087
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   품목명     466088 non-null  object
 1   단위      466088 non-null  object
 2   등급      466088 non-null  object
 3   가격      466088 non-null  int64 
 4   산지      463927 non-null  object
 5   친환경구분   466088 non-null  object
 6   입력일     466088 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 24.9+ MB


In [167]:
# 산지 제외하고는 결측치 없는 것으로 확인
df.isnull().sum()

품목명         0
단위          0
등급          0
가격          0
산지       2161
친환경구분       0
입력일         0
dtype: int64

- 결측치 처리

In [168]:
###
# 양이 많지 않으므로 결측치 제거
df = df.dropna(axis=0)

- 영문명으로 전환

In [169]:
df = df.rename(columns = {'품목명':'prd',
                          '단위':'scale','등급':'class',
                          '가격':'price',
                          '산지':'origin',
                          '친환경구분':'eco',
                          '입력일':'reg_date'})

## column 별 정보

### class

In [170]:
# 이름 모두 영어로 변경
# 9등과 없음 모두 9로 변경
df['class'].value_counts().index

Index(['특(1등)', '상(2등)', '보통(3등)', '9등(등외)', '없음', '5등', '4등', '6등', '7등',
       '8등'],
      dtype='object')

In [171]:
df111 = pd.DataFrame([['특(1등)', 1], ['상(2등)',2], ['보통(3등)',3], ['9등(등외)',9], ['없음',9], ['5등',5], ['4등',4], ['6등',6], ['7등',7],['8등',8]],columns=['class','new_class'])

In [172]:
df2 = df.merge(df111)
df2 = df2.drop(columns='class')

In [173]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[쪽파]쪽파,1kg,2400,전남 보성군,일반,20210331,1
1,[쪽파]쪽파,1kg,2500,충남 아산시,일반,20210331,1
2,[쪽파]쪽파,1kg,2600,충남 아산시,일반,20210331,1
3,[쪽파]쪽파,1kg,2300,전남 보성군,일반,20210331,1
4,[쪽파]쪽파,1kg,2700,전남 보성군,일반,20210331,1
...,...,...,...,...,...,...,...
463922,[양파]양파(일반),15kg,7100,전남 무안군,일반,20210429,8
463923,[기타근채류]기타근채류(기타),15kg,7200,경북 군위군,일반,20210430,8
463924,[기타근채류]기타근채류(기타),15kg,7100,경북 군위군,일반,20210430,8
463925,[양파]양파(일반),15kg,7000,전남 무안군,일반,20210430,8


### eco

In [174]:
# 영문명으로 전환
# 전환기 -> 유기농 재배로 전환중
df2.loc[df2["eco"]=="일반","eco"] = "normal"
df2.loc[df2["eco"]=="우수농산물","eco"] = "good_prd"
df2.loc[df2["eco"]=="무농약","eco"] = "ecofriendly"
df2.loc[df2["eco"]=="유기농","eco"] = "organic"
df2.loc[df2["eco"]=="품질인증","eco"] = "certified"
df2.loc[df2["eco"]=="전환기","eco"] = "ing"

In [175]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[쪽파]쪽파,1kg,2400,전남 보성군,normal,20210331,1
1,[쪽파]쪽파,1kg,2500,충남 아산시,normal,20210331,1
2,[쪽파]쪽파,1kg,2600,충남 아산시,normal,20210331,1
3,[쪽파]쪽파,1kg,2300,전남 보성군,normal,20210331,1
4,[쪽파]쪽파,1kg,2700,전남 보성군,normal,20210331,1
...,...,...,...,...,...,...,...
463922,[양파]양파(일반),15kg,7100,전남 무안군,normal,20210429,8
463923,[기타근채류]기타근채류(기타),15kg,7200,경북 군위군,normal,20210430,8
463924,[기타근채류]기타근채류(기타),15kg,7100,경북 군위군,normal,20210430,8
463925,[양파]양파(일반),15kg,7000,전남 무안군,normal,20210430,8


### reg_date

In [176]:
df2['reg_date'] = df2['reg_date'].astype('str')

In [177]:
df2['reg_date'] = pd.to_datetime(df2['reg_date'].astype('str'),format='%Y%m%d')

In [178]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[쪽파]쪽파,1kg,2400,전남 보성군,normal,2021-03-31,1
1,[쪽파]쪽파,1kg,2500,충남 아산시,normal,2021-03-31,1
2,[쪽파]쪽파,1kg,2600,충남 아산시,normal,2021-03-31,1
3,[쪽파]쪽파,1kg,2300,전남 보성군,normal,2021-03-31,1
4,[쪽파]쪽파,1kg,2700,전남 보성군,normal,2021-03-31,1
...,...,...,...,...,...,...,...
463922,[양파]양파(일반),15kg,7100,전남 무안군,normal,2021-04-29,8
463923,[기타근채류]기타근채류(기타),15kg,7200,경북 군위군,normal,2021-04-30,8
463924,[기타근채류]기타근채류(기타),15kg,7100,경북 군위군,normal,2021-04-30,8
463925,[양파]양파(일반),15kg,7000,전남 무안군,normal,2021-04-30,8


### price

In [179]:
# 계산을 위해 scale 값 미리 변경
# 현재 값들은 string이므로 float 형태로 변환할 필요가 있음
# 몇몇 이상치 처리도 진행
def scale_checker(scale):
  if scale.startswith('.')==True:
    return format(float('0'+scale[:-2]),'.2f')
  else:
    return format(float((scale[:-2])),'.2f')
df2['scale'] = df2['scale'].apply(scale_checker)
df2['scale'] = df2['scale'].astype('float')
# df2['scale'] = format(round(df2['scale'].apply(pd.to_numeric),2),'.2f')

In [180]:
df2['scale'].unique()

array([  1.  ,  10.  ,  11.  ,   8.  ,  12.  ,   6.  ,   4.  ,   5.  ,
        10.8 ,   2.  ,   3.  ,  15.  ,   3.5 ,   2.5 ,   1.5 ,   1.8 ,
        14.  ,  13.  ,  18.  ,  16.  ,  20.  ,   9.  ,  11.5 ,   8.2 ,
        19.  ,  17.  ,   3.6 ,   5.5 ,   6.2 ,  11.3 ,   0.2 ,   5.01,
         0.4 ,   7.2 ,   9.1 ,   7.  ,   0.8 ,  23.  ,  25.  ,   0.5 ,
         6.75,   3.2 ,   4.5 ,   0.3 ,   0.1 ,   0.25,   0.01,   0.05,
         0.13,  22.  ,   3.8 ,   5.1 ,   3.1 ,   6.7 ,   2.2 ,   0.7 ,
         1.4 ,   1.01,   1.2 ,  15.01,   7.5 ,  13.5 ,   0.15,   4.8 ,
        21.  ,  24.  ,   1.6 ,  10.5 ,  10.01,  20.01,   0.6 ,  14.4 ,
         9.6 ,   0.75,   8.01,   4.3 ,   8.3 ,   8.4 ,   6.5 ,  81.  ,
        84.  ,  71.5 ,  46.  ,  73.  ,  47.  ,  76.5 ,  21.5 ,  77.5 ,
        42.  ,  23.5 ,  77.  ,  49.5 ,  14.5 ,   3.9 ,   3.7 ,   4.2 ,
         2.4 ,   3.4 ,  26.  ,   5.8 ,  71.  ,   7.7 ,   7.8 ,   3.3 ,
        31.  ,  14.2 ,  22.68,  11.4 ,   2.9 ,   5.4 ,   8.5 ,   2.3 ,
      

In [181]:
# 1원짜리 거래 발견, 너무 거래값이 작은 값들은 삭제
pd.options.display.float_format = '{:.5f}'.format
df2['price'].describe()

count     463927.00000
mean       21449.14839
std        87238.09868
min           23.00000
25%         7500.00000
50%        14000.00000
75%        26000.00000
max     50420000.00000
Name: price, dtype: float64

In [182]:
df2 = df2.drop(df2[df2['price']<500].index)

In [183]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[쪽파]쪽파,1.00000,2400,전남 보성군,normal,2021-03-31,1
1,[쪽파]쪽파,1.00000,2500,충남 아산시,normal,2021-03-31,1
2,[쪽파]쪽파,1.00000,2600,충남 아산시,normal,2021-03-31,1
3,[쪽파]쪽파,1.00000,2300,전남 보성군,normal,2021-03-31,1
4,[쪽파]쪽파,1.00000,2700,전남 보성군,normal,2021-03-31,1
...,...,...,...,...,...,...,...
463922,[양파]양파(일반),15.00000,7100,전남 무안군,normal,2021-04-29,8
463923,[기타근채류]기타근채류(기타),15.00000,7200,경북 군위군,normal,2021-04-30,8
463924,[기타근채류]기타근채류(기타),15.00000,7100,경북 군위군,normal,2021-04-30,8
463925,[양파]양파(일반),15.00000,7000,전남 무안군,normal,2021-04-30,8


In [184]:
# kg 당 가격 생성

df2['price_kg'] = round((df2['price']/df2['scale']),2)

### prd
- 품목명 정리
- 품목 종류 정리

In [185]:
# 대부분 [상품 종류]상품명 으로 정리되어 있는걸 알 수 있음
df2['prd'].value_counts()

[딸기]설향            13562
[오이]백다다기          13244
[딸기]설향(딸기)         7184
[두릅]두릅(일반)         7103
[참외]참외(기타)         6380
                  ...  
[기타새우국산]기타새우국산        1
[오디]오디(일반)            1
[떫은감]약시               1
[오디]오디(국산)            1
[죽순]죽순                1
Name: prd, Length: 1281, dtype: int64

In [186]:
# 정리를 위해 상품명 삭제
def prd_sort(words):
  p = re.compile('[ㄱ-힣]+')
  word = re.search(p,words)
  return word.group()

df2['prd'] = df2['prd'].apply(prd_sort)

In [187]:
# 기타 품목 삭제
df2 = df2.drop(df2[df2['prd']=='기타'].index)

In [188]:
# 국산,수입산,깐,기타 삭제
def prd_sort2(words):
  p = re.compile('[^국산수입깐기타]+')
  word = re.search(p,str(words))
  return word.group()

df2["prd"] = df2["prd"].apply(prd_sort2)


In [189]:
# 몇몇 품목들 정리가 필요
a = sorted(df2['prd'].unique())
for i in range(len(a)):
  if i%15 != 0:
    print(a[i],end=' ')
  else:
    print(a[i],end=' ')
    print()

가리비 
가물치 가오리 가오리채 가자미 가죽나물 가지 각굴 간재미 갈치 갈치포 감 감귤 감자 갑오징어 갓 
강낭콩 개불 개조개 갯가재 건채류 게 게지 겨자 고 고구마 고구마순 고둥 고들빼 고등어 고비 
고사리 고추 고추잎 곤달비 곤드레나물 골뱅이 곶감 과실류 과일 과일류 과채 굴 귤 근대 근채류 
금감 깻잎 꼬막 꼴뚜 꽁치 꽈리고추 낙지 냉이 노루궁뎅이버섯 논우렁이 농어 눈볼대 느 다랑어 다래 
다래순 다슬 단감 달래 닭새우 당귀 당근 대구 대구고니 대구포 대파 더덕 도다리 도라지 돌나물 
돔 돗나물 동자개 동죽 두류 두릅 두리안 듀리안 딸 땅콩 떫은감 라임 람부탄 레몬 리치 
마 마늘 만가닥 만감 맛 망고 망고스틴 매생이 머위대 메 메론 메밀 명이나물 명태 명태곤니 
명태알 명태포 모시조개 모자반 무 무순 무우 무청 문어 문어바지락 미꾸라지 미나리 미더덕 미역 민들레 
민물새우 바나나 바지락 박 밤 방울토마토 방풍 방풍나물 배 배추 백새우 백합 뱀장어 뱅어 버섯 
버섯류 벅굴 병어 보리 복숭아 봉지굴 부세 부지갱이 부추 붉은고추 붕어 붕장어 브로커리 브로코리 블루베리 
비 비단조개 비름 비트 빈스 사과 살구 삼 삼나물 삼치 상추 새꼬막 새송이 새싹 새우 
새우살 새조개 새치 샐러리 생강 생고추 생채 석류 선동오징어 세러리 셀러리 소라 속새 숙주나물 숫꽃게 
시금치 신선초 실파 쌈채류 쑥 쑥갓 씀바귀 아귀 아보카도 아보카드 아스파라가스 아스파라거스 아욱 알 알배 
암꽃게 야자 양배추 양상추 양송이 양채 양채류 양파 어류 얼갈이 얼갈이배추 엄나무 연근 연어 열무 
염고등어 엽경채류 엽채 엽채류 오가피 오디 오렌지 오만둥이 오이 오징어 오징어몸통 오징어원양 오징어채 완두콩 용과 
우럭조개 우렁 우렁쉥이 우렁이 우뭇가시리 우엉 원추리 유채 임연 잉어 자두 자몽 잔대 재첩 적겨자 
적근대 적새우 적채 전갱이 전복 절단 절단낙지 조개살 죽순 쪽파 쭈꾸미 참나물 참다래 참당귀 참외 
참조 참죽나무순 채소 청경채 청어 체리 총각무 취나물 치커리 칠게 칼라후라워 칼리플라워 

In [190]:
### 예시
# df2[(df2['prd']=='유채') & (df2['price']==4500)]['prd'].unique()
# df2[df2['prd']=='유채']['prd'].unique()

In [191]:
### 원래 단어 뭐였는지 검색
# org_df[org_df['품목명'].str.contains('느')]['품목명'].unique()

In [192]:
### 현재 이름 검색
# df2[df2['prd']=='느']

In [193]:
###
# JG

# 고 -> 고수로 변경
df2.loc[df2["prd"]=="고","prd"] = "고수"
# 과실류, 과일류, 과채, 채소 -> 과채류로 통합
df2.loc[df2["prd"]=="과실류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일야채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일과채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일","prd"] = "과채류"
df2.loc[df2["prd"]=="과채","prd"] = "과채류"
df2.loc[df2["prd"]=="채소","prd"] = "과채류"
# 고들빼 -> 고들빼기로 변경
df2.loc[df2["prd"]=="고들빼","prd"] = "고들빼기"
# 꼴뚜 -> 꼴두기로 변경
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
# 그레이프푸룻 -> 자몽으로 통합
df2.loc[df2["prd"]=="그레이프푸룻","prd"] = "자몽"

# 느 -> 느타리버섯으로 통합
df2.loc[df2["prd"]=="느","prd"] = "느타리버섯"

# 대구 관련 품목들 정리
df2.loc[df2["prd"]=="대구고니","prd"] = "대구"
df2.loc[df2["prd"]=="대구머리","prd"] = "대구"
df2.loc[df2["prd"]=="대구포","prd"] = "대구"
# 듀리안 -> 두리안으로 통합
df2.loc[df2["prd"]=="듀리안","prd"] = "두리안"
# 딸 -> 딸기로 변경
df2.loc[df2["prd"]=="딸","prd"] = "딸기"

# 만가닥 -> 만가닥버섯으로 변경
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
# 맛 -> 맛조개로 변경
df2.loc[df2["prd"]=="맛","prd"] = "맛조개"
# 망고스턴 -> 망고스틴으로 수정
df2.loc[df2["prd"]=="망고스턴","prd"] = "망고스틴"
# 메 -> 메기로 변경
df2.loc[df2["prd"]=="메","prd"] = "메기"
# 명태 관련 품목들 정리
df2.loc[df2["prd"]=="명태곤니","prd"] = "명태"
df2.loc[df2["prd"]=="명태알","prd"] = "명태"
df2.loc[df2["prd"]=="명태포","prd"] = "명태"
# 무우 -> 무로 통합
df2.loc[df2["prd"]=="무우","prd"] = "무"

# 방풍 -> 방풍나물로 통합
df2.loc[df2["prd"]=="방풍","prd"] = "방풍나물"
# 버섯류 -> 버섯으로 변경
df2.loc[df2["prd"]=="버섯","prd"] = "버섯"
# 벗,버찌 -> 체리로 통합
df2.loc[df2["prd"]=="벗","prd"] = "체리"
df2.loc[df2["prd"]=="버찌","prd"] = "체리"
# 브로커리, 브로코리 -> 브로콜리로 통합
df2.loc[df2["prd"]=="브로커리","prd"] = "브로콜리"
df2.loc[df2["prd"]=="브로코리","prd"] = "브로콜리"

# 삼 -> 수삼으로 변경
df2.loc[df2["prd"]=="삼","prd"] = "수삼"
# 새우살 -> 새우로 변경
df2.loc[df2["prd"]=="새우살","prd"] = "새우"
# 세러리, 셀러리 -> 샐러리로 통합
df2.loc[df2["prd"]=="세러리","prd"] = "샐러리"
df2.loc[df2["prd"]=="셀러리","prd"] = "샐러리"
# 생고추 -> 고추로 통합
df2.loc[df2["prd"]=="생고추","prd"] = "고추"

# 아보카드 -> 아보카도로 변경
df2.loc[df2["prd"]=="아보카드","prd"] = "아보카도"
# 아스파라가스 -> 아스파라거스로 변경
df2.loc[df2["prd"]=="아스파라가스","prd"] = "아스파라거스"
# 알 -> 알타리무로 변경
df2.loc[df2["prd"]=="알","prd"] = "알타리무"
# 알배 -> 알배기로 변경
df2.loc[df2["prd"]=="알배","prd"] = "알배기"
# 양채 -> 양채류로 통합
df2.loc[df2["prd"]=="양채","prd"] = "양채류"
# 얼갈이 -> 얼갈이배추로 통합
df2.loc[df2["prd"]=="얼갈이","prd"] = "얼갈이배추"
# 엽채류, 엽채 -> 엽경채류로 통합
df2.loc[df2["prd"]=="엽채류","prd"] = "엽경채류"
df2.loc[df2["prd"]=="엽채","prd"] = "엽경채류"
# 오징어 관련 품목들 정리
df2.loc[df2["prd"]=="오징어다리","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어원양","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채","prd"] = "오징어"
df2.loc[df2["prd"]=="활복오징어","prd"] = "오징어"
# 옥, 풋옥 -> 옥수수로 정리
df2.loc[df2["prd"]=="옥","prd"] = "옥수수"
df2.loc[df2["prd"]=="풋옥","prd"] = "옥수수"
# 임연 -> 임연수어로 정리
df2.loc[df2["prd"]=="임연","prd"] = "임연수어"


# 절단 -> 절단쭈꾸미 -> 쭈꾸미로 정리
df2.loc[df2["prd"]=="절단","prd"] = "쭈꾸미"
# 조개살 -> 조개로 정리
df2.loc[df2["prd"]=="조개살","prd"] = "조개"

# 참조 -> 참조기로 정리
df2.loc[df2["prd"]=="참조","prd"] = "참조기"

# 칼라후라워,칼리플라워, 칼리후라워 -> 컬리플라워로 통합
df2.loc[df2["prd"]=="칼라후라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리플라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리후라워","prd"] = "컬리플라워"
# 코라비 -> 콜라비로 통합
df2.loc[df2["prd"]=="코라비","prd"] = "콜라비"
# 키조개날개 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개날개","prd"] = "키조개"

# 팽이 -> 팽이버섯으로 통합
df2.loc[df2["prd"]=="팽이","prd"] = "팽이버섯"
# 파세리 -> 파슬리로 변경
df2.loc[df2["prd"]=="파세리","prd"] = "파슬리"
# 표고 -> 표고버섯으로 통합
df2.loc[df2["prd"]=="표고","prd"] = "표고버섯"
# 포장바지락 -> 바지락으로 변경
df2.loc[df2["prd"]=="포장바지락","prd"] = "바지락"

### 일단 여기까지 실행
# JY
#각굴 -> 굴로 통합
df2.loc[df2["prd"]=="각굴","prd"] = "굴"
#가오리채 -> 가오리로 통합
df2.loc[df2["prd"]=="가오리채","prd"] = "가오리"
# 게지 -> 키조개로 통합
df2.loc[df2["prd"]=="게지","prd"] = "키조개"
#고 -> 고구마로 통합
df2.loc[df2["prd"]=="고","prd"] = "고구마"
#꼴뚜 -> 꼴뚜기로 통합
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"

#돗나물 -> 돌나물로 통합
df2.loc[df2["prd"]=="돗나물","prd"] = "돌나물"

#레드쉬 -> 레디쉬로 통합
df2.loc[df2["prd"]=="레드쉬","prd"] = "레디쉬"

#만가닥 -> 만가닥버섯으로 통합
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
#메 -> 메기로 통합
df2.loc[df2["prd"]=="메","prd"] = "메기"

#봄동 -> 봄동배추로 통합
df2.loc[df2["prd"]=="봄동","prd"] = "봄동배추"
#봉지굴 -> 굴로 통합
df2.loc[df2["prd"]=="봉지굴","prd"] = "굴"
#박 -> 수박으로 통합
df2.loc[df2["prd"]=="박","prd"] = "수박"
#백조 -> 조기로 통합 
df2.loc[df2["prd"]=="백조","prd"] = "조기"
#벅굴 -> 벚굴로 통합
df2.loc[df2["prd"]=="벅굴","prd"] = "벚굴"
#비 -> 비타민으로 통합
df2.loc[df2["prd"]=="비","prd"] = "비타민"
#빈스 -> 콩으로 통합
df2.loc[df2["prd"]=="빈스","prd"] = "콩"
#삐틀이 -> 고둥으로 통합
df2.loc[df2["prd"]=="삐틀이","prd"] = "고둥"

#새송이 -> 새송이버섯으로 통합
df2.loc[df2["prd"]=="새송이","prd"] = "새송이버섯"
#세트 -> 과채류로 통합
df2.loc[df2["prd"]=="세트","prd"] = "과채류"

#알 -> 알로애로 통합
df2.loc[df2["prd"]=="알","prd"] = "알로에"
#엄나무 -> 음나무로 통합
df2.loc[df2["prd"]=="엄나무","prd"] = "음나무"
#염고등어 -> 고등어로 통합
df2.loc[df2["prd"]=="염고등어","prd"] = "고등어"
#오만둥이 -> 미더덕으로 통합
df2.loc[df2["prd"]=="오만둥이","prd"] = "미더덕"
#오징어 관련 품목들 정리
df2.loc[df2["prd"]=="선동오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어몸통","prd"] = "오징어"
#우렁쉥이 -> 멍게로 통합
df2.loc[df2["prd"]=="우렁쉥이","prd"] = "멍게"
#우렁 -> 우렁이로 통합
df2.loc[df2["prd"]=="우렁","prd"] = "우렁이"
#우뭇가시리 -> 우뭇가사리로 통합
df2.loc[df2["prd"]=="우뭇가시리","prd"] = "우뭇가사리"

#적채 -> 적양배추로 통합
df2.loc[df2["prd"]=="적채","prd"] = "적양배추"
#절단낙지 -> 낙지로 통합
df2.loc[df2["prd"]=="절단낙지","prd"] = "낙지"
#조 -> 조개로 통합
df2.loc[df2["prd"]=="조","prd"] = "조개"

#참다래 -> 키위로 통합
df2.loc[df2["prd"]=="참다래","prd"] = "키위"
#청어원양 -> 청어로 통합
df2.loc[df2["prd"]=="청어원양","prd"] = "청어"

#키조개꼭지 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개꼭지","prd"] = "키조개"

# 다슬 -> 다슬기로 변경
df2.loc[df2["prd"]=="다슬","prd"] = "다슬기"

# 암꽃게, 숫꽃게 -> 꽃게로 통합
df2.loc[df2["prd"]=="암꽃게","prd"] = "꽃게"
df2.loc[df2["prd"]=="숫꽃게","prd"] = "꽃게"
# 우렁 -> 우렁이로 통합
df2.loc[df2["prd"]=="우렁","prd"] = "우렁이"
# 알로애 -> 알로에로 변경
df2.loc[df2["prd"]=="알로애","prd"] = "알로에"
# 피마자잎 -> 피마자로 변경
df2.loc[df2["prd"]=="피미자잎","prd"] = "피마자"
# 조미제품 삭제
df2.drop(df2[df2['prd']=='조미제품'].index,inplace=True)

In [194]:
# 기존 products 리스트와 비교
ex_prd = pd.read_csv("../products.csv")
df2[~df2['prd'].isin(ex_prd['prd'])]['prd'].unique()

array(['가죽나물', '참죽나무순', '호박잎', '오가피', '오디', '고비', '잔대', '다래순', '살구',
       '파파야', '복숭아', '명이나물', '자두', '생채', '다슬기', '비단조개', '꼬막', '농어', '간재미',
       '개불', '전갱이', '갈치포', '새치', '문어'], dtype=object)

In [196]:
# 물품 리스트 최신화
new_prd = pd.DataFrame(df2['prd'].unique(),columns=['prd'])
# products = ex_prd.append(new_prd[~new_prd['prd'].isin(ex_prd['prd'])],ignore_index=True)
products = ex_prd.append(new_prd,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_8672/2619460125.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products = ex_prd.append(new_prd,ignore_index=True)


In [197]:
products.drop(columns=['Unnamed: 0'],inplace=True)
products.drop_duplicates(inplace=True)
products.reset_index(inplace=True,drop=True)

In [198]:
# 다음 작업을 위해 저장
products.to_csv('../products.csv')

In [199]:
# 전체 거래량과 품목량
# 비교를 위해 분포 파악
# 거래량이 너무 적은 품목은 삭제할 필요성이 있음

prd_lst = (df2['prd'].value_counts()).to_frame()
prd_lst.describe()

,prd
count,258.00000
mean,1795.44574
std,5060.11364
min,1.00000
25%,9.25000
50%,90.00000
75%,804.00000
max,45174.00000


In [200]:
# 50개 미만 품목은 비교가 힘드므로 삭제
# 줄어든 종류만 남기기

prd_lst = prd_lst[prd_lst['prd']>=50]
prd_lst.reset_index(inplace=True)
prd_lst = prd_lst.rename(columns = {'index':'prd','prd':'count'})

In [201]:
df3 = df2.merge(prd_lst)
df3[df3['count']>=50]['prd'].value_counts()

딸기     45174
버섯     35233
오이     24290
토마토    23488
고추     18228
       ...  
아귀        56
고둥        56
콩나물       55
겨자        54
패류        52
Name: prd, Length: 147, dtype: int64

In [202]:
# 줄어든 것 확인했으므로 count column은 삭제
df3 = df3.drop(columns='count')

### scale

In [203]:
np.set_printoptions(precision=2, suppress=True)
df3['scale'].unique()

array([  1.  ,  10.  ,  11.  ,   8.  ,  12.  ,   6.  ,   9.  ,  16.  ,
         4.  ,   5.  ,  15.  ,   7.  ,   0.5 ,  13.  ,  14.  ,  17.  ,
         0.8 ,   3.  ,   2.  ,   5.8 ,   0.7 ,  10.8 ,  14.4 ,   9.6 ,
        12.5 ,   0.1 ,   0.2 ,   1.5 ,   2.5 ,   2.4 ,  20.  ,  18.  ,
        23.  ,  22.  ,  19.  ,   3.7 ,  21.  ,   3.5 ,   1.8 ,   0.75,
         0.3 ,   1.2 ,   0.6 ,  14.5 ,  10.01,  13.5 ,   7.5 ,  26.  ,
        24.  ,  28.  ,  11.01,  11.5 ,   3.6 ,   8.2 ,   7.2 ,   8.5 ,
         9.5 ,   7.3 ,   4.5 , 999.  ,   0.4 ,   1.6 ,   8.6 ,   9.8 ,
         4.8 ,   4.2 ,   5.6 ,   6.5 ,   7.7 ,   3.3 ,   6.4 ,   5.5 ,
         6.2 ,  10.7 ,  11.7 ,  12.4 ,  10.6 ,   8.7 ,   7.8 ,   4.4 ,
         8.8 ,  12.6 ,   5.4 ,   8.01,  85.  ,   4.7 ,   6.3 ,  12.2 ,
        11.3 ,   5.01,  71.  ,  50.  ,  45.  ,   3.2 ,   9.1 ,  10.5 ,
         9.07,  22.68,  22.63,  25.  ,  30.  ,   6.75,   3.4 ,  11.4 ,
         2.6 ,   3.8 ,   2.1 ,   3.1 ,  15.2 ,   6.1 ,   8.3 ,   2.7 ,
      

In [204]:
# 50kg 이상 거래된 건들 이상치라 판단하고 확인 진행
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
58513,포도,999.00000,30000,칠레,normal,2021-04-22,1,30.03000
78756,호박,85.00000,16000,경남 진주시,normal,2021-04-10,1,188.24000
92935,가지,71.00000,340800,경남 진주시,certified,2021-04-05,1,4800.00000
93510,가지,50.00000,240000,경남 진주시,certified,2021-04-10,1,4800.00000
427239,오렌지,300.00000,600000,수입산 미국,normal,2021-04-24,1,2000.00000
427310,오렌지,300.00000,590000,수입산 미국,normal,2021-04-26,1,1966.67000
454023,수삼,81.00000,2484000,충남 논산시,normal,2021-04-02,1,30666.67000
454024,수삼,84.00000,2576000,충남 논산시,normal,2021-04-02,1,30666.67000
454025,수삼,71.50000,2192667,충남 논산시,normal,2021-04-02,1,30666.67000
454027,수삼,73.00000,2238667,충남 논산시,normal,2021-04-02,1,30666.67000


In [205]:
df3[(df3['prd']=='오렌지')&(df3['price_kg']<500)]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
425764,오렌지,18.00000,8000,미국,normal,2021-04-02,1,444.44000
425893,오렌지,18.00000,1000,미국,normal,2021-04-05,1,55.56000
425907,오렌지,18.00000,1000,미국,normal,2021-04-05,1,55.56000
425931,오렌지,18.00000,1000,미국,normal,2021-04-05,1,55.56000
426106,오렌지,17.00000,1000,수입산 미국,normal,2021-04-07,1,58.82000
426163,오렌지,18.00000,5000,미국,normal,2021-04-07,1,277.78000
426179,오렌지,18.00000,1000,미국,normal,2021-04-07,1,55.56000
426232,오렌지,17.00000,1000,수입산 미국,normal,2021-04-09,1,58.82000
426309,오렌지,18.00000,1000,미국,normal,2021-04-09,1,55.56000
426343,오렌지,17.00000,1000,수입산 미국,normal,2021-04-10,1,58.82000


In [206]:
# 스케일 오표기된 항목 수정

# 다른 품목은 확인 결과 문제 없음
# 포도 & 호박만 이상치로 판단하고 모두 삭제
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='호박')].index)
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='포도')].index)

In [207]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
92935,가지,71.00000,340800,경남 진주시,certified,2021-04-05,1,4800.00000
93510,가지,50.00000,240000,경남 진주시,certified,2021-04-10,1,4800.00000
427239,오렌지,300.00000,600000,수입산 미국,normal,2021-04-24,1,2000.00000
427310,오렌지,300.00000,590000,수입산 미국,normal,2021-04-26,1,1966.67000
454023,수삼,81.00000,2484000,충남 논산시,normal,2021-04-02,1,30666.67000
454024,수삼,84.00000,2576000,충남 논산시,normal,2021-04-02,1,30666.67000
454025,수삼,71.50000,2192667,충남 논산시,normal,2021-04-02,1,30666.67000
454027,수삼,73.00000,2238667,충남 논산시,normal,2021-04-02,1,30666.67000
454029,수삼,76.50000,2346000,충남 논산시,normal,2021-04-02,1,30666.67000
454031,수삼,77.50000,2376667,충남 논산시,normal,2021-04-02,1,30666.67000


### origin

In [208]:
# 도,시, 국산, 수입 표기 분류 필요
df3['origin'].unique()

array(['전남 보성군', '충남 아산시', '충남 예산군', '충남 보령시', '충남 서천군', '충남 당진시',
       '경기 파주시', '충남 아산시 신창면', '경기 김포시', '경기 여주군', '충남', '제주 제주시',
       '제주 남제주군', '서울 송파구', '충남 부여군', '제주 북제주군', '제주', '제주 서귀포시',
       '제주도 제주시', '제주도 서귀포시', '충남 태안군', '충북 청주시', '전남 무안군', '경기 안성시',
       '충남 아산시 도고면', '충남 서산시', '경기 이천시', '경북 예천군', '강원 인제군', '충남 아산시 둔포면',
       '경기 광주시', '경기 이천시 대월면', '충남 홍성군', '경기도 남양주시', '경기 남양주시', '경기',
       '경기도 김포시', '인천 강화군', '서울시 강서구', '경기 안성시 고삼면', '강원 홍천군', '전북 고창군',
       '전남 신안군', '강원 평창군', '경기 포천시', '경기도 안성시', '경남 밀양시', '경기도 고양시',
       '경기도 광주군', '경기 양평군', '경기 양평군 양동면', '경남 고성군', '전남 고흥군', '전남 해남군',
       '전남 광양시', '뉴질랜드', '수입산 뉴질랜드', '경남 고성군 하일면', '강원 고성군', '수입산 이탈리아',
       '경남 마산시합포구', '경남 통영시 도산면', '경북 성주군', '경북 고령군', '경북 칠곡군', '경북 김천시',
       '대구 달성군', '경북', '충남 논산시', '경북 문경시', '경북 안동시', '경남 남해군 상주면',
       '경남 함안군', '강원 춘천시', '전남 영암군', '서울특별시 동대문구', '경기도 여주군', '경기 용인시',
       '경기도 파주시', '경기 용인시 구성읍', '경기 고양시', '대전 대덕구', '경기 용인시 모현면',
       '경기 평택시', '경

In [209]:
# 해당 조건에 맞는 값 추출

splits = df3['origin'].str.split(' ')
cacul = splits.apply(lambda x: pd.Series(x))
df66 = pd.merge(df3,cacul, left_index=True, right_index=True, how='inner')
df66

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,0,1,2,3
0,쪽파,1.00000,2400,전남 보성군,normal,2021-03-31,1,2400.00000,전남,보성군,NaN,NaN
1,쪽파,1.00000,2500,충남 아산시,normal,2021-03-31,1,2500.00000,충남,아산시,NaN,NaN
2,쪽파,1.00000,2600,충남 아산시,normal,2021-03-31,1,2600.00000,충남,아산시,NaN,NaN
3,쪽파,1.00000,2300,전남 보성군,normal,2021-03-31,1,2300.00000,전남,보성군,NaN,NaN
4,쪽파,1.00000,2700,전남 보성군,normal,2021-03-31,1,2700.00000,전남,보성군,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
461912,멍게,2.00000,65000,경남 통영,normal,2021-04-29,3,32500.00000,경남,통영,NaN,NaN
461913,멍게,5.00000,28000,경남 통영,normal,2021-04-29,3,5600.00000,경남,통영,NaN,NaN
461914,멍게,5.00000,26000,경남 통영,normal,2021-04-29,3,5200.00000,경남,통영,NaN,NaN
461915,멍게,5.00000,25000,경남 통영,normal,2021-04-29,3,5000.00000,경남,통영,NaN,NaN


In [210]:
# 세부 카테고리 정리 (state, city 편집 시 여기서 시작)

df4 = df66.rename(columns = {0:'state',1:'city'})

# 2,3은 필요없는 정보이므로 삭제
df4 = df4.drop(columns=[2,3])

In [211]:
df4['state'].unique()

array(['전남', '충남', '경기', '제주', '서울', '제주도', '충북', '경북', '강원', '경기도', '인천',
       '서울시', '전북', '경남', '뉴질랜드', '수입산', '대구', '서울특별시', '대전', '전라북도',
       '광주', '중국', '부산', '부산시', '경상남도', '강원도', '전주시', '필리핀', '인도네시아',
       '미국', '기타외국', '태국', '인천광역시', '인천시', '칠레', '호주', '오스트레일', '페루',
       '오스트레일리아', '대전시', '광주시', '청주시', '세종', '베트남', '멕시코', '대만', '포항시',
       '성남시', '충북청주시상당구', '전북전주시완산구', '경기고양시일산구', '충청남도', '에콰도르',
       '제주특별자치도', '타이', '부천시', '과테말라', '콜롬비아', '코스타리카', '에쿠아도르', '에콰돌',
       '세종시', '울산', '가락동', '대구시', '제주/서귀포', '스페인', '국내산', '수입', '원양산'],
      dtype=object)

In [212]:
# 시,도 제거
def name_sort(words):
  p = re.compile('[^시도]+')
  word = re.search(p,str(words))
  return word.group()

df4["state"] = df4["state"].apply(name_sort)

# 오타도 제거

def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["state"] = df4["state"].apply(name_sort2)

In [213]:
df4["state"].unique()

array(['전남', '충남', '경기', '제주', '서울', '충북', '경북', '강원', '인천', '전북', '경남',
       '뉴질랜드', '수입산', '대구', '서울특별', '대전', '전라북', '광주', '중국', '부산', '경상남',
       '전주', '필리핀', '인', '미국', '기타외국', '태국', '인천광역', '칠레', '호주', '오스트레일',
       '페루', '오스트레일리아', '청주', '세종', '베트남', '멕', '대만', '포항', '성남', '충북청주',
       '전북전주', '경기고양', '충청남', '에콰', '제주특별자치', '타이', '부천', '과테말라', '콜롬비아',
       '코스타리카', '에쿠아', '에콰돌', '울산', '가락동', '제주/서귀포', '스페인', '국내산', '수입',
       '원양산'], dtype=object)

In [214]:
# state
# 특이케이스 제거

# 국내
df4.loc[df4["state"]=="국내산","city"] = "미상"
df4.loc[df4["state"]=="국내산","state"] = "국산"
df4.loc[df4["state"]=="원양산","city"] = "미상"

## 도단위
df4.loc[df4["state"]=="충청북","state"] = "충북"
df4.loc[df4["state"]=="충청남","state"] = "충남"
df4.loc[df4["state"]=="전라북","state"] = "전북"
df4.loc[df4["state"]=="전라남","state"] = "전남"
df4.loc[df4["state"]=="경상북","state"] = "경북"
df4.loc[df4["state"]=="경상남","state"] = "경남"

## 시단위
### 광역시

df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="대구","city"] = "대구"
df4.loc[df4["state"]=="서울특별","city"] = "서울"
df4.loc[df4["state"]=="서울특별","state"] = "서울"
df4.loc[df4["state"]=="인천광역","city"] = "인천"
df4.loc[df4["state"]=="인천광역","state"] = "인천"
df4.loc[df4["state"]=="부산","city"] = "부산"
df4.loc[df4["state"]=="가락동","city"] = "서울"
df4.loc[df4["state"]=="가락동","state"] = "서울"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="제주","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","state"] = "제주"
df4.loc[df4["state"]=="제주특별자","city"] = "제주"
df4.loc[df4["state"]=="제주특별자","state"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","city"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","state"] = "제주"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="울산","city"] = "울산"
df4.loc[df4["state"]=="울산","state"] = "울산"

### 일반시
df4.loc[df4["state"]=="성남","city"] = "성남"
df4.loc[df4["state"]=="성남","state"] = "경기"
df4.loc[df4["state"]=="전주","city"] = "전주"
df4.loc[df4["state"]=="전주","state"] = "전북"
df4.loc[df4["state"]=="청주","city"] = "청주"
df4.loc[df4["state"]=="청주","state"] = "충북"
df4.loc[df4["state"]=="수원","city"] = "수원"
df4.loc[df4["state"]=="수원","state"] = "경기"
df4.loc[df4["state"]=="부천","city"] = "부천"
df4.loc[df4["state"]=="부천","state"] = "경기"
df4.loc[df4["state"]=="경기고양","city"] = "고양"
df4.loc[df4["state"]=="경기고양","state"] = "경기"
df4.loc[df4["state"]=="충북청주","city"] = "청주"
df4.loc[df4["state"]=="충북청주","state"] = "충북"
df4.loc[df4["state"]=="전북전주","city"] = "전주"
df4.loc[df4["state"]=="전북전주","state"] = "전북"
df4.loc[df4["state"]=="경북/문경","city"] = "문경"
df4.loc[df4["state"]=="경북/문경","state"] = "경북"
df4.loc[df4["state"]=="포항","city"] = "포항"
df4.loc[df4["state"]=="포항","state"] = "경북"


# 외국
## 아시아
df4.loc[df4["state"]=="칠레","city"] = "칠레"
df4.loc[df4["state"]=="칠레","state"] = "수입"
df4.loc[df4["state"]=="일본","city"] = "일본"
df4.loc[df4["state"]=="일본","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="우즈베크","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베크","state"] = "수입"
df4.loc[df4["state"]=="이스라엘","city"] = "이스라엘"
df4.loc[df4["state"]=="이스라엘","state"] = "수입"
df4.loc[df4["state"]=="터키","city"] = "터키"
df4.loc[df4["state"]=="터키","state"] = "수입"
df4.loc[df4["state"]=="중국","city"] = "중국"
df4.loc[df4["state"]=="중국","state"] = "수입"
df4.loc[df4["state"]=="타이","city"] = "태국"
df4.loc[df4["state"]=="타이","state"] = "수입"
df4.loc[df4["state"]=="태국","city"] = "태국"
df4.loc[df4["state"]=="태국","state"] = "수입"
df4.loc[df4["state"]=="필리핀","city"] = "필리핀"
df4.loc[df4["state"]=="필리핀","state"] = "수입"
df4.loc[df4["state"]=="우즈베키스탄","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베키스탄","state"] = "수입"
df4.loc[df4["state"]=="대만","city"] = "대만"
df4.loc[df4["state"]=="대만","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="이란","city"] = "이란"
df4.loc[df4["state"]=="이란","state"] = "수입"
df4.loc[df4["state"]=="러","city"] = "러시아"
df4.loc[df4["state"]=="러","state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"city"] = "인도네시아"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"city"] = "인도"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"state"] = "수입"

## 아메리카

df4.loc[df4["state"]=="캘리포니아","city"] = "미국"
df4.loc[df4["state"]=="캘리포니아","state"] = "수입"
df4.loc[df4["state"]=="포클랜드","city"] = "영국"
df4.loc[df4["state"]=="포클랜드","state"] = "수입"
df4.loc[df4["state"]=="콜롬비아","city"] = "콜롬비아"
df4.loc[df4["state"]=="콜롬비아","state"] = "수입"
df4.loc[df4["state"]=="에콰","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰","state"] = "수입"
df4.loc[df4["state"]=="에쿠아","city"] = "에콰도르"
df4.loc[df4["state"]=="에쿠아","state"] = "수입"
df4.loc[df4["state"]=="에콰돌","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰돌","state"] = "수입"
df4.loc[df4["state"]=="과테말라","city"] = "과테말라"
df4.loc[df4["state"]=="과테말라","state"] = "수입"
df4.loc[df4["state"]=="코스타리카","city"] = "코스타리카"
df4.loc[df4["state"]=="코스타리카","state"] = "수입"
df4.loc[df4["state"]=="멕","city"] = "멕시코"
df4.loc[df4["state"]=="멕","state"] = "수입"
df4.loc[df4["state"]=="페루","city"] = "페루"
df4.loc[df4["state"]=="페루","state"] = "수입"
df4.loc[df4["state"]=="미국","city"] = "미국"
df4.loc[df4["state"]=="미국","state"] = "수입"
df4.loc[df4["state"]=="아르헨티나","city"] = "아르헨티나"
df4.loc[df4["state"]=="아르헨티나","state"] = "수입"

## 유럽
df4.loc[df4["state"]=="스페인","city"] = "스페인"
df4.loc[df4["state"]=="스페인","state"] = "수입"
df4.loc[df4["state"]=="우크라이나","city"] = "우크라이나"
df4.loc[df4["state"]=="우크라이나","state"] = "수입"
df4.loc[df4["state"]=="이탈리아","city"] = "이탈리아"
df4.loc[df4["state"]=="이탈리아","state"] = "수입"
df4.loc[df4["state"]=="불가리아","city"] = "불가리아"
df4.loc[df4["state"]=="불가리아","state"] = "수입"

## 아프리카
df4.loc[df4["state"]=="남아프리카","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카","state"] = "수입"
df4.loc[df4["state"]=="남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카공화국","state"] = "수입"
df4.loc[df4["state"]=="수입남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="수입남아프리카공화국","state"] = "수입"


## 오세아니아
df4.loc[df4["state"]=="뉴질랜드","city"] = "뉴질랜드"
df4.loc[df4["state"]=="뉴질랜드","state"] = "수입"
df4.loc[df4["state"]=="오스트레일","city"] = "호주"
df4.loc[df4["state"]=="오스트레일","state"] = "수입"
df4.loc[df4["state"]=="오스트레일리아","city"] = "호주"
df4.loc[df4["state"]=="오스트레일리아","state"] = "수입"
df4.loc[df4["state"]=="호주","city"] = "호주"
df4.loc[df4["state"]=="호주","state"] = "수입"
df4.loc[df4["state"]=="통가","city"] = "통가"
df4.loc[df4["state"]=="통가","state"] = "수입"
df4.loc[df4["state"]=="뉴칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴칼레","state"] = "수입"
df4.loc[df4["state"]=="뉴","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴","state"] = "수입"
df4.loc[df4["state"]=="누벨칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="누벨칼레","state"] = "수입"
# 기타
df4.loc[df4["state"]=="수입산","state"] = "수입"
df4.loc[df4["state"]=="기타외국","city"] = "미상"
df4.loc[df4["state"]=="기타외국","state"] = "수입"

In [215]:
df4[df4['state']=='nan']

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city


In [216]:
df4['state'].unique()

array(['전남', '충남', '경기', '제주', '서울', '충북', '경북', '강원', '인천', '전북', '경남',
       '수입', '대구', '대전', '광주', '부산', '세종', '울산', '국산', '원양산'],
      dtype=object)

In [217]:
# city
# 우선 결측치 제거
df4.loc[df4["city"].isnull(),"city"] = "미상"

In [218]:
# 이후 시,군 등 제거
def name_sort(words):
  p = re.compile('[^시군]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort)

In [219]:
# \t 도 제거
def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort2)

In [220]:
df4['city'].unique()

array(['보성', '아산', '예산', '보령', '서천', '당진', '파주', '김포', '여주', '미상', '제주',
       '송파구', '부여', '태안', '청주', '무안', '안성', '서산', '이천', '예천', '인제', '광주',
       '홍성', '남양주', '강화', '강서구', '홍천', '고창', '신안', '평창', '포천', '밀양', '고양',
       '양평', '고성', '고흥', '해남', '광양', '뉴질랜드', '이탈리아', '마산', '통영', '성주',
       '고령', '칠곡', '김천', '대구', '논산', '문경', '안동', '남해', '함안', '춘천', '영암',
       '서울', '용인', '대전', '평택', '횡성', '담양', '청양', '완주', '금산', '양주', '완도',
       '양양', '순창', '연천', '양구', '포항', '구리', '양천구', '광명', '계양구', '공주', '의령',
       '진천', '제천', '하남', '광산구', '나주', '북구', '남구', '순천', '여수', '중국', '상주',
       '부산', '김해', '김제', '익산', '옥천', '화성', '창녕', '진안', '진주', '경주', '창원',
       '서구', '장수', '화순', '강진', '장흥', '정읍', '사천', '위', '청도', '무주', '함평',
       '화천', '영광', '임실', '장성', '산', '산청', '함양', '충주', '부안', '구미', '남원',
       '영천', '중구', '철원', '동대문', '양산', '진도', '흥', '구로구', '중랑구', '천안', '전주',
       '은평구', '강릉', '가평', '의성', '강동구', '성남', '필리핀', '인도네', '미국', '태국',
       '인천', '부평구', '옹진', '음성', '칠레', '호주', '페루',

In [226]:
# 나머지 케이스 정리

# 국내
df4.loc[df4["city"]=="기타","city"] = "미상"
## 일반시
df4.loc[df4["city"]=="위","city"] = "군위"
df4.loc[df4["city"]=="고흥(나로도)","city"] = "고흥"
df4.loc[df4["city"]=="산","city"] = "군산"
df4.loc[df4["city"]=="장항(서천)","city"] = "서천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="순천(광양)","city"] = "순천"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="화성(평택)","city"] = "화성"
df4.loc[df4["city"]=="흥","city"] = "시흥"
df4.loc[df4["city"]=="목포(무안)","city"] = "목포"
df4.loc[df4["city"]=="목포(무안)","state"] = "전남"
df4.loc[df4["city"]=="보성(벌교)","city"] = "보성"
df4.loc[df4["city"]=="미금","city"] = "성남"
df4.loc[df4["city"]=="순","city"] = "순창"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="남"),"city"] = "남원"
df4.loc[(df4['prd']=="호박") & (df4['state']=='전북'),"city"] = "전주"
## 광역시
df4.loc[df4["city"]=="강서구","city"] = "서울"
df4.loc[df4["city"]=="은평구","city"] = "서울"
df4.loc[df4["city"]=="강동구","city"] = "서울"
df4.loc[df4["city"]=="마포구","city"] = "서울"
df4.loc[df4["city"]=="중구","city"] = "서울"
df4.loc[df4["city"]=="송파구","city"] = "서울"
df4.loc[df4["city"]=="도봉구","city"] = "서울"
df4.loc[df4["city"]=="중랑구","city"] = "서울"
df4.loc[df4["city"]=="양천구","city"] = "서울"
df4.loc[df4["city"]=="성동구","city"] = "서울"
df4.loc[df4["city"]=="강남구","city"] = "서울"
df4.loc[df4["city"]=="동대문구","city"] = "서울"
df4.loc[df4["city"]=="영등포구","city"] = "서울"
df4.loc[df4["city"]=="영등포","city"] = "서울"
df4.loc[df4["city"]=="서초구","city"] = "서울"
df4.loc[df4["city"]=="구로구","city"] = "서울"
df4.loc[df4["city"]=="동대문","city"] = "서울"
df4.loc[df4["city"]=="금천구","city"] = "서울"

df4.loc[df4["city"]=="광산구","city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="남구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="북구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="서구"),"city"] = "광주"
df4.loc[df4["city"]=="동구","city"] = "광주"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="남구"),"city"] = "인천"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="서구"),"city"] = "인천"
df4.loc[df4["city"]=="미추홀","city"] = "인천"
df4.loc[df4["city"]=="부평구","city"] = "인천"
df4.loc[df4["city"]=="남동구","city"] = "인천"
df4.loc[df4["city"]=="미추홀구","city"] = "인천"
df4.loc[df4["city"]=="계양구","city"] = "인천"
df4.loc[df4["city"]=="마산(고성,진해)","city"] = "창원"
df4.loc[(df4["state"]=="부산")&(df4["city"]=="북구"),"city"] = "부산"
df4.loc[df4["city"]=="오포","city"] = "부산"
df4.loc[df4["city"]=="부산진","city"] = "부산"
df4.loc[df4["city"]=="사상구","city"] = "부산"
df4.loc[df4["city"]=="부산진구","city"] = "부산"
df4.loc[df4["city"]=="북제주","city"] = "제주"
df4.loc[df4["city"]=="남제주","city"] = "제주"

# 해외
## 아시아
df4.loc[df4["city"]=="러","city"] = "러시아"
df4.loc[df4["city"]=="말레이지아","city"] = "말레이시아"
df4.loc[df4["city"]=="타이","city"] = "태국"
df4.loc[df4["city"]=="인도네","city"] = "인도네시아"
## 아메리카
df4.loc[df4["city"]=="멕","city"] = "멕시코"
## 아프리카
df4.loc[df4["city"]=="남아공산","city"] = "남아공"
## 유럽
## 오세아니아

In [227]:
# 기존 city 리스트와 비교
ex_city = pd.read_csv("../cities.csv")
df4[~df4['city'].isin(ex_city['city'])]['city'].unique()

array(['과천', '울릉', '동두천', '모로코'], dtype=object)

In [228]:
# 리스트 최신화
new_city = pd.DataFrame(df4['city'].unique(),columns=['city'])
cities = ex_city.append(new_city,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_8672/1856950528.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = ex_city.append(new_city,ignore_index=True)


In [229]:
cities.drop(columns=['Unnamed: 0'],inplace=True)
cities.drop_duplicates(inplace=True)
cities.reset_index(inplace=True,drop=True)

In [231]:
cities.to_csv('../cities.csv')

In [232]:
# 처리 완료하였으니 df4는 백업으로 두고 df5 활용
df5 = df4.drop(columns='origin')

# 전처리 결과

# 파일 저장

In [233]:
df5.to_csv('../data_proceed/202104_proceed.csv')